In [ ]:
import os.path
import datetime
import pygetm.legacy
import pygetm.input
import pygetm.airsea
import cartopy.crs as ccrs

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
igotm_data = next(filter(os.path.isdir, ('/server/data', '../../../igotm/data')))

northsea_path = os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc')  # path to legacy GETM North Sea test case
era_path = os.path.join(igotm_data, 'ERA-interim/20??.nc')              # path to ERA NetCDF file with t2m variable

map_proj = ccrs.PlateCarree()                                           # map projection to use for plotting

# Default time for plots
start = dt = datetime.datetime(2016,4,1,12)

In [ ]:
# Load GETM domain so we have a lon/lat grid to compute on/interpolate to.
domain = pygetm.legacy.domain_from_topo(northsea_path, nlev=1)
domain.initialize(1)

#tcc = domain.T.array(long_name='cloud cover', units='1', fill=0.2)

# Load meteorology from ERA
tcc = domain.T.map(pygetm.input.get_from_nc(era_path, 'tcc', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
t2m = domain.T.map(pygetm.input.get_from_nc(era_path, 't2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
d2m = domain.T.map(pygetm.input.get_from_nc(era_path, 'd2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
sp = domain.T.map(pygetm.input.get_from_nc(era_path, 'sp', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
u10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'u10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
v10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'v10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))

In [ ]:
# Compute and plot humidity
t2m.update(dt)
d2m.update(dt)
sp.update(dt)

es, ea, qs, qa, rhoa = pygetm.airsea.humidity(domain.T, 3, d2m - 273.15, sp, t2m - 273.15, t2m - 273.15)

fig, (ax1, ax2, ax3) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=3, figsize=(16, 2.5))
(100*ea/es).plot(ax=ax1)
qa.plot(ax=ax2)
rhoa.plot(ax=ax3)
ax1.coastlines()
ax2.coastlines()
ax3.coastlines()


In [ ]:
# Compute and plot longwave radiation
tcc.update(dt)

ql = pygetm.airsea.longwave_radiation(domain.T, 1, t2m, t2m, tcc, ea, qa)

fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
ql.plot(ax=ax)
ax.coastlines()

In [ ]:
# Compute and plot albedo
angle = pygetm.airsea.solar_zenith_angle(dt, domain.T)

fig, (ax1, ax2) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=2, figsize=(12, 3.5))
pygetm.airsea.albedo_water(dt, domain.T, 1, angle).plot(ax=ax1)
pygetm.airsea.albedo_water(dt, domain.T, 2, angle).plot(ax=ax2)
ax1.coastlines()
ax2.coastlines()

In [ ]:
# Compute and plot heat and momentum fluxes
u10.update(dt)
v10.update(dt)
taux, tauy, qe, qh = pygetm.airsea.airsea_fluxes(domain.T, 1, t2m, t2m, u10, v10, rhoa, qs, qa)

fig, ((ax1, ax2), (ax3, ax4)) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, nrows=2, ncols=2, figsize=(12, 6))
taux.plot(ax=ax1)
tauy.plot(ax=ax2)
qe.plot(ax=ax3)
qh.plot(ax=ax4)
for ax in (ax1, ax2, ax3, ax4):
    ax.coastlines()

In [ ]:
# Update cloud cover to start time and compute shortwave radiation.
tcc.update(start)
angle = pygetm.airsea.solar_zenith_angle(start, domain.T)
swr = pygetm.airsea.shortwave_radiation(start, domain.T, angle, tcc)

# Set up plot of cloud cover and shortwave radiation at start time - we'll animate it later
fig, (ax1, ax2) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=2, figsize=(12, 3))
pc_cloud = tcc.plot(ax=ax1, vmin=0, vmax=1)
pc_swr = swr.plot(ax=ax2, vmin=0, vmax=750)
ax1.coastlines()
ax2.coastlines()
title = fig.suptitle(start)

In [ ]:
# Animate cloud cover and shortwave radiation
for i in range(24 * 4 * 10):
    dt = start + datetime.timedelta(seconds=(i + 1) * 900)
    tcc.update(dt)
    pygetm.airsea.solar_zenith_angle(dt, domain.T, out=angle)
    pygetm.airsea.shortwave_radiation(dt, domain.T, angle, tcc, out=swr)
    pc_cloud.set_array(tcc.values.ravel())
    pc_swr.set_array(swr.values.ravel())
    title.set_text(dt)
    fig.canvas.draw()